# Exercise 2
### In this Exercise, you are given a dataset with K=200 unique categories, so each datapoint is a category between 1 and 200. Your task is to calculate the parameters, theta, of the underlying multinomial distribution, using MLE and MAP.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('data.csv')
data = df['category'].values
K = 200 

# usually this is not done in practice since this kind of split is biased by order of data! 
split_at = int(len(data) * 0.8) 
train = data[:split_at]
val = data[split_at:]

print(f'K: {K}')
print(f'seen categories K\': {np.unique(train).size}')
print(f'N_train: {len(train)}, N_val: {len(val)}')

K: 200
seen categories K': 164
N_train: 400, N_val: 100


In [13]:

def counts_from_data(data, K):
    """
    Count occurrences per category, assuming categories are 1..K.
    Returns a length-K array of counts in order 1..K.
    """
    # bincount with minlength K+1 to include index 0; drop index 0
    counts = np.bincount(data, minlength=K+1)[1:]
    if counts.shape[0] != K:
        raise ValueError("Counts length mismatch with K. Check category labels and K.")
    return counts.astype(np.float64)


In [14]:
def mle(data, K):
    """
    MLE for multinomial parameters theta over K categories (1..K).
    theta_k = n_k / N
    """
    counts = counts_from_data(data, K)
    N = counts.sum()
    if N == 0:
        raise ValueError("Empty data for MLE.")
    theta_mle = counts / N
    return theta_mle


In [15]:
theta_mle = mle(train, K)
assert np.isclose(np.sum(theta_mle), 1.0)
print(f"Amount of category probabilities equal to zero (MLE): {(theta_mle==0).sum()} of {K}")


Amount of category probabilities equal to zero (MLE): 37 of 200


In [17]:
def map_estimate(data, K, alpha):
    """
    MAP estimate for multinomial with symmetric Dirichlet(alpha) prior.
    Mode of Dirichlet posterior: (n_k + alpha - 1) / (N + K*(alpha - 1)),
    but when alpha <= 1 and n_k == 0, the mode is on boundary at 0.
    here implement this by clipping negative numerators to 0 and renormalizing.
    """
    if alpha is None:
        raise ValueError("alpha must be provided for MAP.")
    counts = counts_from_data(data, K)
    N = counts.sum()

    # Numerator for MAP mode
    numerators = counts + (alpha - 1.0)

    # If alpha <= 1 and some counts are zero, the MAP mode is on boundary (0) for those k.
    # Clipping to 0 implements the boundary solution correctly.
    numerators = np.maximum(numerators, 0.0)

    # Sum of numerators:
    s = numerators.sum()
    if s == 0:
        # This would happen only in degenerate cases; fall back to uniform
        theta_map = np.full(K, 1.0 / K, dtype=np.float64)
    else:
        theta_map = numerators / s

    # For alpha > 1 and no clipping, theta_map matches the closed form denominator (N + K*(alpha-1))
    # because sum(n_k + alpha - 1) = N + K*(alpha - 1).
    return theta_map




In [18]:

# Choose a value for alpha: (using >1 to avoid zeros; alpha=1 equals MLE)
# A mild smoothing often works well, e.g., 1.1 or 1.01.

ALPHA = 1.1
theta_map = map_estimate(train, K, alpha=ALPHA)
assert np.isclose(np.sum(theta_map), 1.0)
print(f"Amount of category probabilities equal to zero (MAP, alpha={ALPHA}): {(theta_map==0).sum()} of {K}")

Amount of category probabilities equal to zero (MAP, alpha=1.1): 0 of 200


In [10]:
def log_likelihood(data, theta):
    """
    Compute log-likelihood of data under multinomial parameters theta.
    Using counts: sum_k n_k * log(theta_k).
    Adds small epsilon to avoid log(0).
    """
    eps = 1e-12
    # Map categories (1..K) to indices (0..K-1)
    K = theta.shape[0]
    counts = counts_from_data(data, K)
    ll = float(np.sum(counts * np.log(theta + eps)))
    return ll



In [11]:
# Evaluate
ll_mle_train = log_likelihood(train, theta_mle)
ll_mle_val = log_likelihood(val, theta_mle)

ALPHA_MLE = 1  # MAP with alpha=1 should equal MLE (see explanation below)
theta_map_alpha1_val = map_estimate(train, K, alpha=ALPHA_MLE)
ll_map_alpha1_val = log_likelihood(val, theta_map_alpha1_val)

ll_map = log_likelihood(val, theta_map)
ll_map_train = log_likelihood(train, theta_map)
ll_map_val = log_likelihood(val, theta_map)

ALPHA=1 on the validation set should give the same log likelihood as the MLE (can you explain why?)

In [12]:
print(f"[TRAIN] Log-Likelihood MLE: {ll_mle_train:.2f}")
print(f"[TRAIN] Log-Likelihood MAP (alpha={ALPHA}): {ll_map_train:.2f}")
print(f"[VAL]   Log-Likelihood MLE: {ll_mle_val:.2f}")
print(f"[VAL]   Log-Likelihood MAP with alpha=1: {ll_map_alpha1_val:.2f}")
print(f"[VAL]   Log-Likelihood MAP with alpha={ALPHA}: {ll_map_val:.2f}")                          

[TRAIN] Log-Likelihood MLE: -1968.25
[TRAIN] Log-Likelihood MAP (alpha=1.1): -1971.90
[VAL]   Log-Likelihood MLE: -810.67
[VAL]   Log-Likelihood MAP with alpha=1: -810.67
[VAL]   Log-Likelihood MAP with alpha=1.1: -559.68


Choosing α (ALPHA):

α = 1: no smoothing; equal to MLE; zero probabilities for unseen categories. It reduces to the MLE, which is purely based on observed frequencies.Therefore, the probability estimates and the resulting log-likelihoods are identical.
α > 1: additive smoothing; reduces overfitting to the training set; often improves validation likelihood/robustness.
α ∈ [1.01,2] is a reasonable range to try first. Using the provided alpha sweep to pick what maximizes validation log-likelihood.
MAP with α=1 does not add any smoothing or prior information.
